In [122]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt

df = pd.read_csv("Walmart.csv")

df['transaction_date'] = pd.to_datetime(df['transaction_date'], errors='coerce')

conn = sqlite3.connect(":memory:")  # or use 'walmart.db' for file

df.to_sql("walmart_data", conn, index=False, if_exists="replace")


5000

In [268]:
query = """
SELECT 
    strftime('%Y-%m', transaction_date) AS month,
    SUM(quantity_sold) AS total_quantity_sold
FROM walmart_data
GROUP BY month
ORDER BY month;
"""

monthly_sales = pd.read_sql(query, conn)

monthly_sales['month'] = pd.to_datetime(monthly_sales['month'], format='%Y-%m')

monthly_sales.to_csv("monthly_quantity_sold.csv", index=False)

print(monthly_sales)


       month  total_quantity_sold
0 2024-01-01                 1672
1 2024-02-01                 1674
2 2024-03-01                 1809
3 2024-04-01                 1668
4 2024-05-01                 1768
5 2024-06-01                 1613
6 2024-07-01                 1833
7 2024-08-01                 1968
8 2024-09-01                  909


In [270]:
query = """
SELECT 
    strftime('%Y-%m', transaction_date) AS month,
    category,
    SUM(quantity_sold) AS total_quantity_sold
FROM walmart_data
GROUP BY month, category
ORDER BY month;
"""
category_sales = pd.read_sql(query, conn)

category_sales['month'] = pd.to_datetime(category_sales['month'], format='%Y-%m')

category_sales.to_csv("monthly_sales_by_category.csv", index=False)

print(category_sales)


        month     category  total_quantity_sold
0  2024-01-01   Appliances                  802
1  2024-01-01  Electronics                  870
2  2024-02-01   Appliances                  855
3  2024-02-01  Electronics                  819
4  2024-03-01   Appliances                  853
5  2024-03-01  Electronics                  956
6  2024-04-01   Appliances                  767
7  2024-04-01  Electronics                  901
8  2024-05-01   Appliances                  874
9  2024-05-01  Electronics                  894
10 2024-06-01   Appliances                  732
11 2024-06-01  Electronics                  881
12 2024-07-01   Appliances                  900
13 2024-07-01  Electronics                  933
14 2024-08-01   Appliances                  951
15 2024-08-01  Electronics                 1017
16 2024-09-01   Appliances                  435
17 2024-09-01  Electronics                  474


In [130]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

mae = mean_absolute_error(df['actual_demand'], df['forecasted_demand'])
mse = mean_squared_error(df['actual_demand'], df['forecasted_demand'])
rmse = mse ** 0.5
mape = (abs(df['actual_demand'] - df['forecasted_demand']) / df['actual_demand']).mean() * 100

print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")

Mean Absolute Error (MAE): 137.52
Mean Squared Error (MSE): 28286.88
Root Mean Squared Error (RMSE): 168.19
Mean Absolute Percentage Error (MAPE): 59.93%


In [278]:
query = """
SELECT 
    strftime('%Y-%m', transaction_date) AS month,
    SUM(ABS(actual_demand - forecasted_demand)) AS total_forecast_error
FROM walmart_data
GROUP BY month
ORDER BY month;
"""

error_df = pd.read_sql(query, conn)

error_df['month'] = pd.to_datetime(error_df['month'], format='%Y-%m')

error_df.to_csv("monthly_forcast_error.csv", index=False)

print(error_df)


       month  total_forecast_error
0 2024-01-01                 76944
1 2024-02-01                 77291
2 2024-03-01                 86000
3 2024-04-01                 77477
4 2024-05-01                 84947
5 2024-06-01                 75173
6 2024-07-01                 81025
7 2024-08-01                 88950
8 2024-09-01                 39771


In [282]:
df['transaction_date'] = pd.to_datetime(df['transaction_date'])
df['month'] = df['transaction_date'].dt.to_period('M')
df['days_in_month'] = df['transaction_date'].dt.days_in_month

query = """ 
    SELECT
    product_name,
    strftime('%Y-%m', transaction_date) AS month,
    SUM(quantity_sold) AS quantity_sold,
    AVG(inventory_level) AS inventory_level,
    MAX(strftime('%d', date(transaction_date, 'start of month', '+1 month', '-1 day'))) AS days_in_month
    FROM walmart_data
    GROUP BY product_name, month
    ORDER BY product_name, month;
 """
monthly_demand = pd.read_sql(query, conn)

monthly_demand['days_in_month'] = monthly_demand['days_in_month'].astype(int)

monthly_demand['daily_demand'] = (
    monthly_demand['quantity_sold'] / monthly_demand['days_in_month']
).round()


safety_stock = 25

monthly_demand['reorder_trigger_day'] = ((monthly_demand['inventory_level'] - safety_stock) / monthly_demand['daily_demand']).round()

monthly_demand.to_csv("reorder_timing_by_product.csv", index=False)


print(monthly_demand[['product_name', 'month', 'quantity_sold', 'daily_demand', 'inventory_level', 'reorder_trigger_day']].head(12))


   product_name    month  quantity_sold  daily_demand  inventory_level  \
0        Camera  2024-01            225           7.0       228.041096   
1        Camera  2024-02            245           8.0       283.240964   
2        Camera  2024-03            191           6.0       277.621622   
3        Camera  2024-04            145           5.0       248.865385   
4        Camera  2024-05            217           7.0       257.878378   
5        Camera  2024-06            248           8.0       255.395062   
6        Camera  2024-07            267           9.0       245.975309   
7        Camera  2024-08            234           8.0       271.391892   
8        Camera  2024-09            101           3.0       227.083333   
9        Fridge  2024-01            233           8.0       275.486111   
10       Fridge  2024-02            190           7.0       260.507937   
11       Fridge  2024-03            231           7.0       260.024096   

    reorder_trigger_day  
0          

In [286]:
query = """
SELECT month, product_name, MAX(total_quantity_sold) AS max_quantity_sold
FROM (
    SELECT 
        strftime('%Y-%m', transaction_date) AS month,
        product_name,
        SUM(quantity_sold) AS total_quantity_sold
    FROM walmart_data
    GROUP BY month, product_name
) AS monthly_totals
GROUP BY month
ORDER BY month;
"""

best_selling = pd.read_sql(query, conn)

best_selling.to_csv("best_selling_products.csv", index=False)

print(best_selling)


     month     product_name  max_quantity_sold
0  2024-01  Washing Machine                235
1  2024-02           Camera                245
2  2024-03           Tablet                327
3  2024-04           Tablet                245
4  2024-05           Fridge                247
5  2024-06           Camera                248
6  2024-07       Smartphone                276
7  2024-08               TV                305
8  2024-09           Fridge                171


In [288]:
query = """
SELECT 
    product_name,
    SUM(quantity_sold) AS total_quantity_sold,
    AVG(unit_price) AS avg_unit_price,
    SUM(unit_price * quantity_sold) AS total_revenue
FROM walmart_data
GROUP BY product_name
ORDER BY total_revenue DESC;
"""
product_profitability = pd.read_sql(query, conn)

top_products = product_profitability.head(10)

top_products.to_csv("profitable_products.csv", index=False)

print(top_products)


      product_name  total_quantity_sold  avg_unit_price  total_revenue
0               TV                 1926     1052.523066     2049493.86
1           Tablet                 1964      996.608253     1996253.02
2           Fridge                 1967      993.185954     1938012.69
3       Smartphone                 1876     1038.293276     1931310.04
4  Washing Machine                 1807     1052.249837     1897934.02
5           Camera                 1873     1015.638312     1895104.13
6       Headphones                 1816     1022.561726     1846334.45
7           Laptop                 1685     1018.981159     1709159.24


In [248]:
query = """
SELECT 
    strftime('%Y-%m', transaction_date) AS month,
    SUM(unit_price * quantity_sold) AS total_sales
FROM walmart_data
GROUP BY month
ORDER BY month;
"""

monthly_revenue = pd.read_sql(query, conn)

monthly_revenue['month'] = pd.to_datetime(monthly_revenue['month'])
monthly_revenue.to_csv("monthly_revenue_obtained.csv", index=False)

print(monthly_revenue)


       month  total_sales
0 2024-01-01   1731651.65
1 2024-02-01   1767062.38
2 2024-03-01   1833450.84
3 2024-04-01   1739800.75
4 2024-05-01   1786559.47
5 2024-06-01   1600978.97
6 2024-07-01   1878513.51
7 2024-08-01   2018315.81
8 2024-09-01    907268.07


In [216]:
query = """
SELECT 
    strftime('%Y-%m', transaction_date) AS month,
    SUM(forecasted_demand) AS forecasted_demand,
    SUM(actual_demand) AS actual_demand
FROM walmart_data
GROUP BY month
ORDER BY month;
"""

sales_comparison = pd.read_sql_query(query, conn)

sales_comparison['month'] = pd.to_datetime(sales_comparison['month'])
print(sales_comparison)

sales_comparison.to_csv("monthly_forecast_vs_actual.csv", index=False)

       month  forecasted_demand  actual_demand
0 2024-01-01             167788         166932
1 2024-02-01             171914         169799
2 2024-03-01             181400         183080
3 2024-04-01             165247         169430
4 2024-05-01             172057         176216
5 2024-06-01             160850         170451
6 2024-07-01             184297         178184
7 2024-08-01             196832         193476
8 2024-09-01              85285          87874


In [223]:
df['revenue'] = df['unit_price'] * df['quantity_sold']

query = """
SELECT 
    store_id,
    SUM(unit_price * quantity_sold) AS total_revenue
FROM walmart_data
GROUP BY store_id
ORDER BY total_revenue DESC;
"""

revenue_store = pd.read_sql_query(query, conn)

revenue_store.to_csv("revenue_by_store.csv", index=False)

print(revenue_store)

    store_id  total_revenue
0          1      847018.97
1         11      831769.36
2          5      830071.66
3         17      822912.10
4          9      813169.46
5          2      796410.56
6         10      791735.58
7         20      789721.39
8         15      781436.41
9          8      775187.40
10        19      750516.46
11        16      739372.93
12         3      738115.04
13        14      726846.56
14        13      722058.89
15         7      720463.92
16        18      719431.61
17        12      704529.53
18         4      700333.70
19         6      662499.92


In [234]:
query = """
SELECT 
    store_location,
    SUM(quantity_sold * unit_price) AS total_sales
FROM walmart_data
GROUP BY store_location
ORDER BY total_sales DESC;
"""

region_sales = pd.read_sql_query(query, conn)

region_sales.to_csv("region_sales.csv", index=False)

print(region_sales)

    store_location  total_sales
0  Los Angeles, CA   3276299.63
1      Chicago, IL   3156726.82
2     New York, NY   2964077.24
3        Miami, FL   2962567.02
4       Dallas, TX   2903930.74


In [243]:
query = """
SELECT 
    weekday,
    COUNT(*) AS transaction_count
FROM walmart_data
GROUP BY weekday
ORDER BY 
    CASE weekday
        WHEN 'Monday' THEN 1
        WHEN 'Tuesday' THEN 2
        WHEN 'Wednesday' THEN 3
        WHEN 'Thursday' THEN 4
        WHEN 'Friday' THEN 5
        WHEN 'Saturday' THEN 6
        WHEN 'Sunday' THEN 7
    END;
"""

weekday_counts = pd.read_sql_query(query, conn)

weekday_counts.to_csv("weekday_transaction_counts.csv", index=False)

print(weekday_counts)

     weekday  transaction_count
0     Monday                775
1    Tuesday                736
2  Wednesday                691
3   Thursday                760
4     Friday                668
5   Saturday                710
6     Sunday                660
